In [1]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Tue_May__3_18:49:52_PDT_2022
Cuda compilation tools, release 11.7, V11.7.64
Build cuda_11.7.r11.7/compiler.31294372_0


/home/djjin/anaconda3/envs/conda_visual_HPE/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


torch:  1.13 ; cuda:  cu117
detectron2: 0.6


In [2]:
from detectron2.utils.logger import setup_logger
setup_logger()

import os, sys
home_dir = os.path.abspath(os.getcwd()+"/../")
sys.path.append(home_dir)
home_dir = os.path.abspath(os.getcwd())
sys.path.append(home_dir)
print(home_dir)

from utils.arguments import load_vcoco_opt_command
from utils.distributed import init_distributed

/home/djjin/Mygit/X-Decoder/notebooks


Invalid MIT-MAGIC-COOKIE-1 key

In [3]:
opt, cmdline_args = load_vcoco_opt_command(home_dir)
opt = init_distributed(opt)

In [4]:
from datasets.registration.register_vcoco_dataset import register_all_vcoco
_root = os.getenv("DATASET", "../datasets")
register_all_vcoco(_root)

datasets/ADE20K_2021_17_01/images_detectron2/training
datasets/ADE20K_2021_17_01/images_detectron2/validation


In [5]:
from datasets.dataset_mappers import VCOCODatasetMapper
from datasets.build import build_detection_train_loader
from datasets.build import _train_loader_from_config
from xdecoder.utils import configurable
from detectron2.data.common import DatasetFromList, MapDataset
from detectron2.data.samplers import TrainingSampler
from detectron2.data.build import build_batch_data_loader

In [6]:
@configurable(from_config=_train_loader_from_config)
def build_hoi_train_loader(
    dataset, *, mapper, sampler=None, total_batch_size, aspect_ratio_grouping=True, num_workers=0
):
    if isinstance(dataset, list):
        dataset = DatasetFromList(dataset, copy=False)
    if mapper is not None:
        dataset = MapDataset(dataset, mapper)
    if sampler is None:
        sampler = TrainingSampler(len(dataset))
    assert isinstance(sampler, torch.utils.data.sampler.Sampler)
    return build_batch_data_loader(
        dataset,
        sampler,
        total_batch_size,
        aspect_ratio_grouping=aspect_ratio_grouping,
        num_workers=num_workers,
    )

In [7]:
opt['DATASETS']['Train'] = ['vcoco_train']
dataset_names = opt['DATASETS']['Train']
opt.update(opt['VCOCO'])
mapper = VCOCODatasetMapper(opt, True)
dataloaders = [build_hoi_train_loader(opt, dataset_names, mapper=mapper)]
dataloaders

[07/12 17:56:48 d2.data.common]: Serializing 5400 elements to byte tensors and concatenating them all ...
[07/12 17:56:48 d2.data.common]: Serialized dataset takes 3.62 MiB


In [8]:
from datasets.build import build_train_dataloader


In [13]:
train_data_loader = [build_train_dataloader(opt)]

[07/12 18:08:59 d2.data.common]: Serializing 5400 elements to byte tensors and concatenating them all ...
[07/12 18:08:59 d2.data.common]: Serialized dataset takes 3.62 MiB


In [12]:
import cv2
from datasets.utils.vcoco_utils import valid_obj_ids, get_coco_instance_ID_to_name, random_color
from matplotlib import pyplot as plt

for dataloader, dataset_name in zip(train_data_loader, dataset_names):
    dataloader = list(dataloader)
    print(len(dataloader))
    with torch.no_grad():
        for idx, batch in enumerate(dataloader):
            if idx == 3:
                break
            for test in batch:
                target_img = test["image"]
                target = test["instances"]
                img = target_img.permute(1, 2, 0).detach().cpu().numpy()
                hh, ww = img.shape[0], img.shape[1]
                # print(hh, ww)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

                obj_labels = []
                labels = []
                verb_names = []
                ob_boxes = []
                sub_boxes = []

                obj_boxes = target['boxes']
                for test in range(obj_boxes.shape[0]):
                    cx, cy, w, h = obj_boxes[test]
                    color = random_color()
                    cx, cy, w, h = cx * ww, cy * hh, w * ww, h * hh
                    n_box = list(map(int, [cx - 0.5 * w, cy - 0.5 * h, cx + 0.5 * w, cy + 0.5 * h]))
                    x1, y1, x2, y2 = n_box
                    ob_boxes.append(n_box)

                for idx in target['labels']:
                    label_name = get_coco_instance_ID_to_name(valid_obj_ids[idx])
                    labels.append(label_name)

                for label, box in zip(labels, ob_boxes):
                    color = random_color()
                    x1, y1, x2, y2 = box
                    cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)

                    x = (x1 + x2) / 2
                    y = (y1 + y2) / 2

                    plt.scatter(x, y, 30, color=[i/255 for i in color])
                    plt.text(
                        x + 5,
                        y + 5,
                        label,
                        fontsize=10,
                        bbox=dict(facecolor=[i/255 for i in color], alpha=0.5),
                        )

                plt.imshow(img)
                plt.show()

KeyboardInterrupt: 